In [ ]:
import sys
sys.path.append("..")

In [ ]:
from src.utils import dataset_utils, model_utils

device = "cuda:4"
batch_size = 32
layer = 15
mt = model_utils.load_model("gpt2-xl", fp16=True)
mt.model.to(device);

In [ ]:
dataset = dataset_utils.load_dataset("counterfact", split="train[:5000]")

In [ ]:
from src import editors, precompute
import importlib
importlib.reload(editors)
importlib.reload(precompute)
layer = 10
splits = dataset_utils.maybe_train_test_split(dataset, test_size=.1)
splits = precompute.editor_inputs_from_dataset(
    mt=mt,
    dataset=splits,
    batch_size=batch_size,
    layers=[layer],
    device=device)

In [ ]:
from src import editors

import importlib
importlib.reload(editors)

editor = editors.BiaffineEditor(mt=mt, layer=layer).to(device)
editor.fit(
    dataset=splits,
    device=device,
    batch_size=batch_size,
    lr=1e-4,
    max_epochs=5,
    lam_adv=None,
    lam_kl=None,
#     lam_kl=.2,
)

In [ ]:
importlib.reload(editors)
entity = "Jane Smith"
attribute = "has an MD from Harvard Medical School"
context = "{entity} {attribute}".format(entity=entity, attribute=attribute)
prompt = "{entity} works in a".format(entity=entity)

import torch

with editors.apply(editor, alpha=1, device=device) as edited:
    outputs = edited.model.generate({
        "entity": [entity],
        "context": [context],
        "prompt": [prompt],
        "attribute": [attribute],
    }, max_new_tokens=20)
print(mt.tokenizer.batch_decode(outputs))

In [ ]:
importlib.reload(editors)
# importlib.reload(tokenizer_utils)
editor.half()
results = editor.evaluate(splits["test"], alpha=1, batch_size=batch_size, device=device)

In [ ]:
result = results.results[2]
print(result.sample["target_mediated"], "vs", result.sample["target_unmediated"])
print(result.before_generations)
print(result.before_top_tokens)
print(result.after_generations)
print(result.after_top_tokens)

# Pretrained Editor

In [ ]:
from src.utils import dataset_utils, model_utils

device = "cuda:5"
mt = model_utils.load_model("gpt2-xl", fp16=False)
mt.model.to(device);

In [ ]:
from src import editors

import torch



In [ ]:
experiment = "linear_with_loss_terms"

entity = "Jane"
prompt = "wrote the famous song"
attribute = "is the lead singer of the band Nirvana"

for layer in range(25):
    state_dict = torch.load(
        f"../results/{experiment}/linear/{layer}/weights.pth",
        map_location=device)
    editor = editors.LinearEditor(mt=mt, layer=layer,
                                  use_entity=False,
                                  edit_last_entity_token=True)
    editor.load_state_dict(state_dict)
    editor.to(device)

    with editors.apply(editor, device=device) as mt_edit:
        outputs = mt_edit.model.generate({
            "entity": entity,
            "prompt": f"{entity} {prompt}",
            "context": f"{entity} {attribute}",
            "attribute": attribute,
        }, max_new_tokens=20)
    print(layer, mt.tokenizer.batch_decode(outputs))